In [1]:
import cv2
import dlib
from contextlib import contextmanager
import numpy as np

@contextmanager
def video_capture(*args, **kwargs):
    cap = cv2.VideoCapture(*args, **kwargs)
    try:
        yield cap
    finally:
        cap.release()

def yield_images():
    # capture video
    with video_capture(0) as cap:
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        while True:
            # get video frame
            ret, img = cap.read()

            if not ret:
                raise RuntimeError("Failed to capture image")

            yield img


image_generator = yield_images()
detector = dlib.get_frontal_face_detector()
img_size = 224
margin = 0.4

In [2]:
for img in image_generator:
    input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)

    # detect faces using dlib detector
    detected = detector(input_img, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))

    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1], (img_size, img_size))

        # predict ages and genders of the detected faces
        # results = model.predict(faces)
        # predicted_genders = results[0]
        # ages = np.arange(0, 101).reshape(101, 1)
        # predicted_ages = results[1].dot(ages).flatten()

        # draw results
        # for i, d in enumerate(detected):
        #     label = "{}, {}".format(int(predicted_ages[i]),
        #                             "M" if predicted_genders[i][0] < 0.5 else "F")
        #     draw_label(img, (d.left(), d.top()), label)

    cv2.imshow("result", img)
    key = cv2.waitKey(30)

    if key == 27:  # ESC
        break


KeyboardInterrupt: 